In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease/heart.csv


*This is my first kaggle submission...*

**So if you think I made any mistake I'm all ears!
Any suggestions from your side will be highly appreciated**

In [2]:
dataset = pd.read_csv('/kaggle/input/heart-disease/heart.csv')
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
dataset.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [4]:
categorical = ['sex', 'cp', 'restecg', 'slope', 'thal']
do_not_touch = ['fbs', 'exang']
non_categorical = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']

# OneHotEncoding categorical values

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),categorical)],remainder='passthrough')
X = ct.fit_transform(dataset[categorical+do_not_touch+non_categorical])
y = dataset['target'].values

In [6]:
X[0,:]

array([  0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,   0. ,
         1. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
        63. , 145. , 233. , 150. ,   2.3,   0. ])

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=1)

# As you see, last 6 columns of the dataset are non categorical values.. 
# So they need to be scaled

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[:,-6:] = scaler.fit_transform(X_train[:,-6:])
X_test[:,-6:] = scaler.transform(X_test[:,-6:])

In [9]:
X_train[0,:]

array([ 0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        , -0.61990074, -0.08877873,
        0.37511601,  0.91545786, -0.37805012, -0.70686683])

In [10]:
from sklearn.svm import SVC
estimator = SVC()

parameters = [{'kernel':['rbf'],
               'C':[1,10,100,1000],
               'gamma':[1,0.1,0.001,0.0001],
            },
            {'kernel':['poly'],
               'C':[1,10,100,1000],
               'gamma':[1,0.1,0.001,0.0001],
             'degree':range(1,5)}
             ]



# Using Grid Search to find best fit SVC model 

In [11]:
from sklearn.model_selection import GridSearchCV
 
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=True
)


In [12]:
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    4.5s
[Parallel(n_jobs=10)]: Done 610 tasks      | elapsed:    6.7s
[Parallel(n_jobs=10)]: Done 800 out of 800 | elapsed:   23.1s finished


SVC(C=100, gamma=0.0001)

In [13]:
y_pred = grid_search.best_estimator_.predict(X_test)

So, this is the final confusion matrix and the accuracy score

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[13  3]
 [ 0 15]]


0.9032258064516129